In [149]:
function display_position(pos_matrix, cars, num_lanes, time, stop)
    street = []
    for _ in 1:num_lanes
        lane = collect("-" ^ stop)
        lane[stop] = '|'
        lane[stop-30] = '|'
        push!(street, lane)
    end
    
    for c in 1:length(cars)
        lane = street[cars[c][2]]
        lane[pos_matrix[c, time]] = Char('0' + c)
    end
    
    for l in street
        println(join(l))
    end
end

display_position (generic function with 2 methods)

In [150]:
function make_cars(num_cars)
    car = [-1, 1, 1, 1, false]                                                 # start lead car at pos = -1
    cars = [car]
    for _ in 1:num_cars-1
        car = []
        arrived = false
        speed = rand(1:3)                                               # randomize speed    
        spacing = rand(2:4)                                             # randomize spacing
        lane = 1                                                        # all cars start in lane 1
        pos = cars[end][1] - 1 - spacing                                # next car trails the previous car by spacing
        car = [pos, lane, spacing, speed, arrived]
        push!(cars, car)
    end

    pos_of_last_car = cars[end][1]                                      # get pos of last car
    shift_pos = 1 - pos_of_last_car
    for car in cars
        car[1] += shift_pos                                             # shift pos, last car starts at pos = 1, negative numbers are awkward
    end
    
    return cars
end

make_cars (generic function with 2 methods)

In [151]:
function simulate_traffic(cars, num_lanes, dest, time)
    
    num_cars = length(cars)

    # Simulate Traffic
    pos_matrix = zeros(Int, num_cars, time)                             # record pos of each car over time
    stop = dest + 30                                                    # cars will stop at the traffic light

    for c in 1:num_cars
        pos_matrix[c, 1] = cars[c][1]                                   # t = 1
    end

    #println("t = 1")
    #display_position(pos_matrix, cars, num_lanes, 1, stop)
    println()
    
    total_lane_switches = 0
    arrival_times = zeros(Int, num_cars)
    for t in 2:time
        #println("t = ", t)
        for car_id in 1:length(cars)
            car = cars[car_id]
            speed = car[4]

            move = true

            for j in 1:length(cars)
                other_car = cars[j]
                dist = other_car[1] - car[1]
                if dist > 0                                            # check if car is in front
                    if (dist - speed) <= car[3]                        # if car is closer than (spacing + speed) car does not move
                        if other_car[2] == car[2]                      # check if car is in same lane

                            switch_down = true
                            if car[2] <= 1
                                switch_down = false
                            end
                            for c in cars
                            #   if other_car is in next lane AND is within + OR - 1 of curr_car pos: switch_lane = false
                                if c[2] == car[2]-1 && (c[1] == car[1] || c[1] == car[1]+1 || c[1] == car[1]+1)
                                    switch_down = false
                                end
                            end

                            switch_up = true
                            if car[2] >= num_lanes
                                switch_up = false
                            end
                            for c in cars
                            #   if other_car is in next lane AND is within + OR - 1 of curr_car pos: switch_lane = false
                                if c[2] == car[2]+1 && (c[1] == car[1] || c[1] == car[1]+1 || c[1] == car[1]+1)
                                    switch_up = false
                                end
                            end

                            if !switch_up && !switch_down
                                move = false                          # damn they got me boxed in >:{
                            elseif switch_up
                                car[2] += 1                           # switch up one lane
                                total_lane_switches += 1
                            else
                                car[2] -= 1                           # switch down one lane
                                total_lane_switches += 1
                            end

                        end
                    end
                end
            end

            if car[1] + speed > stop                           # if car is at a traffic light car does not move
                move = false
                pos_matrix[car_id, t] = car[1]
                continue
            end

            if move
                car[1] += speed                                        # car moves per speed
            end
            
            if car[5] == false && car[1] >= dest
                car[5] = true
                arrival_times[car_id] = t
            end 

            pos_matrix[car_id,t] = car[1]                              # save pos for car c at time t
        end

        #display_position(pos_matrix, cars, num_lanes, t, stop)
        #println()
    end
    
    return arrival_times, total_lane_switches
    
end

simulate_traffic (generic function with 2 methods)

In [152]:
cars = make_cars(6)

6-element Vector{Vector{Int64}}:
 [21, 1, 1, 1, 0]
 [18, 1, 2, 2, 0]
 [13, 1, 4, 3, 0]
 [10, 1, 2, 1, 0]
 [5, 1, 4, 1, 0]
 [1, 1, 3, 3, 0]

In [156]:
function reset_cars!(cars)
    
    cars[1][1] = -1
    cars[1][2] = 1
    cars[1][5] = false
    
    for c in 2:length(cars)
        car = cars[c]
        car[1] = cars[c-1][1] - 1 - car[3]
        car[2] = 1
        car[5] = false
    end
    
    pos_last_car = cars[end][1]                                      # get pos of last car
    shift_pos = 1 - pos_last_car
    for car in cars
        car[1] += shift_pos                                             # shift pos, last car starts at pos = 1, negative numbers are awkward
    end
    
end

reset_cars! (generic function with 1 method)

In [164]:
cars = make_cars(10)

for lanes in 1:5
    
    arrivals, lane_switches = simulate_traffic(cars, lanes, 50, 500)
    
    first_arrival = minimum(arrivals)
    last_arrival = maximum(arrivals)
    
    println(arrivals)
    println(lane_switches)
    
    reset_cars!(cars)
    println(cars)
end


[11, 15, 19, 25, 28, 35, 39, 0, 0, 0]
0
[[40, 1, 1, 1, 0], [36, 1, 3, 1, 0], [33, 1, 2, 3, 0], [28, 1, 4, 3, 0], [23, 1, 4, 3, 0], [18, 1, 4, 1, 0], [14, 1, 3, 1, 0], [9, 1, 4, 2, 0], [5, 1, 3, 2, 0], [1, 1, 3, 1, 0]]

[11, 15, 7, 9, 10, 33, 37, 22, 24, 50]
28
[[40, 1, 1, 1, 0], [36, 1, 3, 1, 0], [33, 1, 2, 3, 0], [28, 1, 4, 3, 0], [23, 1, 4, 3, 0], [18, 1, 4, 1, 0], [14, 1, 3, 1, 0], [9, 1, 4, 2, 0], [5, 1, 3, 2, 0], [1, 1, 3, 1, 0]]

[11, 15, 7, 9, 10, 33, 37, 22, 24, 50]
49
[[40, 1, 1, 1, 0], [36, 1, 3, 1, 0], [33, 1, 2, 3, 0], [28, 1, 4, 3, 0], [23, 1, 4, 3, 0], [18, 1, 4, 1, 0], [14, 1, 3, 1, 0], [9, 1, 4, 2, 0], [5, 1, 3, 2, 0], [1, 1, 3, 1, 0]]

[11, 15, 7, 9, 10, 33, 37, 22, 24, 50]
53
[[40, 1, 1, 1, 0], [36, 1, 3, 1, 0], [33, 1, 2, 3, 0], [28, 1, 4, 3, 0], [23, 1, 4, 3, 0], [18, 1, 4, 1, 0], [14, 1, 3, 1, 0], [9, 1, 4, 2, 0], [5, 1, 3, 2, 0], [1, 1, 3, 1, 0]]

[11, 15, 7, 9, 10, 33, 37, 22, 24, 50]
48
[[40, 1, 1, 1, 0], [36, 1, 3, 1, 0], [33, 1, 2, 3, 0], [28, 1, 4, 3, 0], [2